In [1]:
from copy import deepcopy
from collections import Counter, defaultdict

In [2]:
data = open("input/14.txt").read().splitlines()

# Part 1

In [3]:
class Poly:
    def __init__(self, seq, codes):
        self.seq = seq
        self.codes = codes
        
    def run(self):
        inserts = []
        for code in self.codes:
            first, second = code.split(' -> ')
            for idx in range(len(self.seq) - 1):
                chunk = self.seq[idx: idx + 2]
                if chunk == first:
                    inserts.append([idx, second])
        for count, (idx, char) in enumerate(sorted(inserts)):
            cur_idx = idx + count + 1
            self.seq = self.seq[:cur_idx] + char + self.seq[cur_idx:]   

In [4]:
p = Poly(data[0], data[2:])
for _ in range(10):
    p.run()

In [5]:
counts = Counter(list(p.seq)).values()
part1 = max(counts) - min(counts)
print(f"Answer #1: {part1}")

assert part1 == 2891

Answer #1: 2891


# Part 2

In [6]:
class SmartPoly:
    def __init__(self, seq, codes):
        self.seq = seq
        self.codes = codes
        self.code_map = defaultdict(int)
        for idx in range(len(self.seq) - 1):
            chunk = self.seq[idx: idx + 2]
            self.code_map[chunk] += 1
        
        # We need to keep track of the start and end chunk
        # In the end we multiply the occurrences by two,
        # but the start/end value shouldn't be counted twice
        self.start_chunk = self.seq[:2]
        self.end_chunk = self.seq[-2:]
        
    def run(self):
        changes = []
        for code in self.codes:
            first, second = code.split(' -> ')
            count = self.code_map.get(first)
            if not count:
                continue
            new1 = first[0] + second
            new2 = second + first[1]
            changes.append([new1, True, count])
            changes.append([new2, True, count])
            changes.append([first, False, count])
            
            # Update chunks
            if self.start_chunk == first:
                self.start_chunk = new1
            if self.end_chunk == first:
                self.end_cunk = new2
            
        for chunk, status, count in changes:
            if status:
                self.code_map[chunk] += count
            else:
                self.code_map[chunk] -= count

In [7]:
p = SmartPoly(data[0], data[2:])

for _ in range(40):
    p.run()

In [8]:
counts = defaultdict(int)
for pair, count in p.code_map.items():
    if count == 0:
        continue
    counts[pair[0]] += count
    counts[pair[1]] += count
    # Take the start/end into account
    # because we count everything twice
    # (and divide by two) add 1 extra
    if pair == p.start_chunk:
        counts[pair[0]] += 1
    if pair == p.end_chunk:
        counts[pair[1]] += 1

res = [val // 2 for val in counts.values()]
part2 = max(res) - min(res)
print(f"Answer #2: {part2}")

assert part2 == 4607749009683

Answer #2: 4607749009683
